In [4]:
import ee
from datetime import datetime, timedelta, date
from termcolor import colored
import folium
from time import time

In [ ]:
#ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
# Alerts mapbiomas
alerts_mapbiomas = ee.FeatureCollection('users/valderli/mapbiomas-alertas-consolidados-2018-2020')
grid_sentinel = ee.FeatureCollection('users/valderli/grade_sentinel_brasil')
comissao_box = ee.FeatureCollection('users/valderli/comissao_box')

# bandas que serão exportatadas
bands = ['B2', 'B3', 'B4', 'B8', 'pB2', 'pB3', 'pB4', 'pB8', 'mask']

In [7]:
# folium
def addLayer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.addLayer = addLayer

In [8]:
def readSentinel2(point):
    return ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(point)

In [9]:
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask);

# função fornecida pelo Marco Rosa
def maskS2clouds2(image):
    qa = image.select('QA60');
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = pow(2, 10);
    cirrusBitMask = pow(2, 11);

    # clear if both flags set to zero.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
    mask = clear.eq(1); 

    mask_pixels_claros = image.select('B1').lt(1000);
    mask_pixels_escuro = image.select('B8').gt(1000);

    return image.updateMask(mask).updateMask(mask_pixels_claros).updateMask(mask_pixels_escuro);  

In [10]:
def showMap(centroid, sentinel2):
    lon, lat = centroid.getInfo()['coordinates']

    test = (sentinel2
    .filterDate('2019-07-01', '2019-07-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90))
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .map(maskS2clouds2)
    .first()
    )

    # Define a map centered on southern Maine.
    map_s2 = folium.Map(location=[lat, lon], zoom_start=9)

    # Add the image layer to the map and display it.
    map_s2.addLayer(
        test, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'first')
    display(map_s2)

In [11]:
# Define function of buffering and bounding      
def bounding_box(feature):
    buffer = feature.buffer(500)  # buffer in m
    box = buffer.bounds()
    return(box)

In [12]:
def readScene(scene):
    grid_scene = grid_sentinel.filterMetadata('Name', 'equals', scene);
    return grid_scene.geometry()

In [13]:
def readAlertsByScene(grid_scene, period):
    
    # datas de antes do alerta ser mapeado e depois de ser mapeado
    date_start = datetime.strptime(period['start'], '%d/%m/%Y').timestamp() * 1000
    date_end = datetime.strptime(period['end'], '%d/%m/%Y').timestamp() * 1000

    def intersectionScene(feature):
        intersection = feature.intersection(grid_scene, ee.ErrorMargin(1));
        return intersection.set({ 'length'  : intersection.geometry().geometries().length()});

    alerts_scene = alerts_mapbiomas.map(intersectionScene);
    alerts_scene = alerts_scene.filterMetadata('length', 'greater_than', 0);

    #alerts_scene = alerts_mapbiomas.filterBounds(grid_sentinel_scene)

    alerts_scene = (alerts_scene
                        .filterMetadata('beforeDate', 'greater_than', date_start)
                        .filterMetadata('afterDate', 'less_than', date_end)
                       )
   
    return alerts_scene

In [14]:
def millisecondsTodate(m_seconds):
    return datetime.fromtimestamp(m_seconds / 1000.0).strftime('%Y-%m-%d')

In [15]:
def exportToCloudStorage(image, name):
    task = ee.batch.Export.image.toCloudStorage(
        image = image.select(bands).toInt(),
        description = name,
        bucket = 'tensorflow-fire-cerrado1',
        fileNamePrefix = 'images_train_test_alerts_deforestation/{0}'.format(name),
        maxPixels = 1e13,
        scale = 10,
        region = None
    );
    task.start()
    
def exportToDrive(image, folder, name):
    task = ee.batch.Export.image.toDrive(
        image = image.select(bands).toInt(),
        folder=folder, # exportando para "dataset_samples_deforestation/images_to_samples_raw"
        description = name,
        maxPixels = 1e13,
        scale = 10,
        region = None
    );
    task.start()

In [16]:
# IRECI (B7 − B4) / (B5 / B6) 
def normalizedDifferenceIRECI(image):
    b4 = image.select('B4');
    b5 = image.select('B5');
    b6 = image.select('B6');
    b7 = image.select('B7');
    return b7.subtract(b4).divide(b5.divide(b6)).rename('IRECI');

In [17]:
def checkIntersection(sample_box, alerts_scene):
    def intersectionSampleBoxAnotherSample(feature):
        intersection = feature.intersection(sample_box, ee.ErrorMargin(1));
        return intersection.set({ 'length'  : intersection.geometry().geometries().length()});
    
    return alerts_scene.map(intersectionSampleBoxAnotherSample);

In [18]:
# cria uma nova data a partir de uma data
# adicionando dias ou subtraindo
def newDateFromDate(date, days):
    date, time = date.split("T")

    data_formated = datetime.strptime(date, '%Y-%m-%d')

    return data_formated + timedelta(days=days)

In [18]:
def renderExport(alerts_scene, sentinel2):
    
    alerts_features = alerts_scene.getInfo()['features']
    
    # total de alertas
    total_alerts = len(alerts_features)
    print(colored('\nTotal de aletas {}\n'.format(total_alerts), 'green'))

    # boxes dos alertas/amostras
    alerts_boxes = alerts_scene.map(bounding_box);
    alerts_boxes_features = alerts_boxes.getInfo()['features']
    
    total_exported = 0
    total_not_exported = 0
    
    for index, alert_box in enumerate(alerts_boxes_features):
        # properties
        props = alert_box['properties']

        before_date = props['beforeDate'] # antes de desmatar
        after_date  = props['afterDate'] # depois de desmatado
        alert_id    = props['IDAlerta']
        
        sample_box = ee.Feature(alert_box)

        #area = sample_box.geometry().perimeter(1)
        #print('area', area.getInfo())

        # Verifica a intersecção da "sample_box" com todas as amostras,
        # se intersectar apenas uma vez então exporta a imagem.
        # Isso evita exportar uma imagem com desmatamentos próximos
        # um do outro que seja de amostras diferentes. 
        # Garantindo que não vai ter área que foi desmatada 
        # e que não tem a máscara da amostra
        intersect = checkIntersection(sample_box, alerts_scene)
        intersect = intersect.filterMetadata('length', 'greater_than', 0);
        intersect = intersect.getInfo()['features']

        if(len(intersect) == 1):
            # image before alert
            before_end = before_date - (1 * m_day)
            before_start = before_end - (10 * m_day)

            before_date_start = millisecondsTodate(before_start)
            before_date_end = millisecondsTodate(before_end)

            before_image = (sentinel2
                            .filterDate(before_date_start, before_date_end)
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                            .sort('CLOUDY_PIXEL_PERCENTAGE')
                            .map(maskS2clouds2)
                            .first()
                           )
           
            # image after alert
            after_start = after_date + (1 * m_day)
            after_end = after_start + (10 * m_day)

            after_date_start = millisecondsTodate(after_start)
            after_date_end = millisecondsTodate(after_end)

            after_image = (sentinel2
                           .filterDate(after_date_start, after_date_end)
                           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
                           .sort('CLOUDY_PIXEL_PERCENTAGE')
                           .map(maskS2clouds2)
                           .first()
                          )

            # merge images before and after alert
            image = (after_image
                     .addBands(before_image.select('B2').rename('pB2'))
                     .addBands(before_image.select('B3').rename('pB3'))
                     .addBands(before_image.select('B4').rename('pB4'))
                     .addBands(before_image.select('B8').rename('pB8'))
                    )

            # mask
            alert = alerts_features[index]
            feature_alert = ee.Feature(alert)
            mask = (ee.Image(1).clip(feature_alert)
                    .unmask(ee.Image.constant(0))
                    .select(['constant'], ['mask'])
                   )

            image = image.addBands(mask.select('mask'));

            # clip image
            image = image.select(bands).clip(sample_box)

            try:
                # se o getInfo falhar vai cair no except
                # o que indica que tem algum problema com a imagem anterior
                # e ou a imagem posterior ao alerta de desmatamento
                image.getInfo()

                has_image = len(image.geometry().getInfo()['coordinates'])

                # mesmo entrando no try, para exportar a imagem,
                # fez-se necessário verificar se a imagem existe
                if has_image:
                    # export image
                    name = 'alert_id_{0}_{1}'.format(alert_id, int(time()))
                    exportToDrive(image, name)

                    print(colored('Exportado: imagem do alerta com id {0}, alerta idenditicado entre {1} e {2}'
                            .format(alert_id, before_date_end, after_date_start), 'green'))
                    total_exported += 1
                else:
                    print(colored('Não exportado: alerta/amostra sem imagem disponível.', 'red'))
                    total_not_exported += 1

            except:
                print(colored('Não exportado: alerta/amostra sem imagem disponível.', 'red'))
                total_not_exported += 1
        else:
            print(colored('Não exportado: alerta/amostra, "sample_box" intersecta com {0} amostras'
                          .format(len(intersect)), 'yellow'))
            total_not_exported += 1
            
    print(colored('\nExportado: Total {0}, exportou {1} e não exportou {2}.'
                    .format(total_alerts, total_exported, total_not_exported), 'blue'))
    
    return total_exported, total_not_exported

In [21]:
# cenas que terão as amostras exportadas
scenes = [
    # -- organiar --
    #'23MKQ', '23MLQ', '23MMQ',
    #'23MMP', 
    #'23MMN', '23MNN', '23MPN',
    #'23MNP', '23MPP', '23MNQ', '23MPQ', '23MLP', '23MLN',
    #'23MLM', '23MMM', '23MNM', '23MKM', '23MKP', '23MKN',
    #'22MHU',
    #'22MHT',
    #'22MGS', '22MHS',
    # -- #### --
    #'22LFR', '22LGR', '22LHR', '23LKL', '23LLL', '23LML', '23LNL',
    #'22LEQ', '22LFQ', '22LGQ', '22LHQ', '23LKK', '23LLK', '23LMK', '23LNK',
    #'22LDP', '22LEP', '22LFP', '22LGP', '22LHP', '23LLJ', '23LMJ', '23LNJ', '23LPJ',
    #'22LDN', '22LEN', '22LFN', '22LGN', '22LHN', '23LKH', '23LLH', '23LMH', '23LNH', '23LPH',
    '21LUG', '21LVG', '21LWG', '21LXG', '21LYG', '22LBM', '22LCM', #'22LDM', '22LEM', '22LFM', '22LGM', '22LHM', '23LKG', '23LLG', '23LMG', '23LNG', '23LPG',
    '21LTF', '21LUF', '21LVF', '21LWF', '21LXF', '21LYF', '21LZF', '22LBL', '22LCL', '22LDL', '22LEL', '22LFL', '22LGL', '22LHL', '23LKF', '23LLF', '23LMF', '23LNF', '23LPF', 
    '21LTE', '21LUE', '21LVE', '21LWE', '21LXE', '21LYE', '22LBK', '22LCK', '22LDK', '22LEK', '22LFK', '22LGK', '22LHK', '23LKE', '23LLE', '23LME', '23LNE', '23LPE',
]

period_month = {
    'start': '01/07/',
    'end': '31/08/'
}

years = [2019, 2020]

In [ ]:
# um dia em milissegundos
m_day = 8.64e+7

total_exported_all = 0
total_not_exported_all = 0

for year in years:
       
    period = {
        'start': period_month['start'] + str(year),
        'end': period_month['end'] + str(year)
    }
    
    for index, scene in enumerate(scenes):

        print(colored('\n\nExpotando alertas do ano {}, da cena: {}'.format(year, scene), 'cyan', attrs=['bold']))

        # lê a geometria da cena
        grid_scene = readScene(scene)

        # contróide da cena
        centroid = grid_scene.centroid()

        # lê a imagem correpondente a cena
        sentinel2 = readSentinel2(centroid)

        # mostra a imagem no mapa
        #showMap(centroid, sentinel2)

        # lê os alerts do mapbiomas correpondente a cena
        alerts_scene = readAlertsByScene(grid_scene, period)

        # prepara as imagens e expota
        total_exported, total_not_exported = renderExport(alerts_scene, sentinel2)

        total_exported_all += total_exported
        total_not_exported_all += total_not_exported
        
print(colored(
        '\nExportação finalizada. Total {0}, exportou {1} e não exportou {2}.'
        .format(
            total_exported_all + total_not_exported_all,
            total_exported_all,
            total_not_exported_all
        ), 'blue', attrs=['bold']))

# Exportando amostras com erro de classificação

#### São amostras coletadas com base na classificação, ou seja, olhando o resultado da classifição são coletadas amostras onde teve erro de classificação

In [ ]:
comissao_box_features = comissao_box.getInfo()['features']

total_exported = 0
total_not_exported = 0

# total de amostras
total_samples = len(comissao_box_features)
print(colored('\nTotal de amostras {}\n'.format(total_samples), 'green'))

for index, c_box in enumerate(comissao_box_features):
        
    sample_box = ee.Feature(c_box)

    # properties
    props = c_box['properties']

    date_before = props['date_before'] # data da imagem anterior
    date_after  = props['date_after'] # data da imagem posterior

    # contróide da cena
    centroid = sample_box.geometry().centroid()

    #print(centroid)

    # lê a imagem correpondente a cena
    sentinel2 = readSentinel2(centroid)

    # image before
    date_before_end = newDateFromDate(date_before, 1)
    before_image = sentinel2.filterDate(date_before, date_before_end).first()
               
    # image after
    date_after_end = newDateFromDate(date_after, 1)
    after_image = sentinel2.filterDate(date_after, date_after_end).first()

    # merge images before and after
    image = (after_image
             .addBands(before_image.select('B2').rename('pB2'))
             .addBands(before_image.select('B3').rename('pB3'))
             .addBands(before_image.select('B4').rename('pB4'))
             .addBands(before_image.select('B8').rename('pB8'))
            )

    # mask
    mask = ee.Image(0).select(['constant'], ['mask'])

    image = image.addBands(mask.select('mask'));

    # clip image
    image = image.select(bands).clip(sample_box)

    has_image = len(image.geometry().getInfo()['coordinates'])

    if has_image:
        # export image
        folder = 'images_error_classify_raw'
        name = 'sample_{0}'.format(int(time()))
        exportToDrive(image, folder, name)

        print(colored('Exportado imagem/amostra {0} de {1}'.format(index+1, total_samples), 'green'))
        total_exported += 1
    else:
        print(colored('Não exportado: sem imagem disponível.', 'red'))
        total_not_exported += 1

print(colored('\nExportado: Total {0}, exportou {1} e não exportou {2}.'
                    .format(total_samples, total_exported, total_not_exported), 'blue'))